In [8]:
import pandas as pd

# Read the CSV files
df_billing = pd.read_csv('data/session_billing_events.csv')
df_dau = pd.read_csv('data/logsM.ai_daily_active_users.csv')
df_workflow = pd.read_csv('data/builder.workflow_executions_logs.csv')

print("df_billing:", df_billing.shape)
print(df_billing.head())
print("\ndf_dau:", df_dau.shape)
print(df_dau.head())
print("\ndf_workflow:", df_workflow.shape)
print(df_workflow.head())


df_billing: (1000, 5)
   companyId    month conversationType  event_count  billable_count
0          1  2025-06          UTILITY            4               4
1          1  2025-07   MARKETING_LITE        16451           16451
2          1  2025-07          UTILITY         2653            2358
3          1  2025-08   MARKETING_LITE        20387           20387
4          1  2025-08          UTILITY         3722            3291

df_dau: (1652, 4)
   dau_count  companyId    month  unique_users_count
0         14          1  2025-05                  14
1          6          5  2025-01                   6
2        126          5  2025-02                  62
3        165          5  2025-03                  78
4        143          5  2025-04                  62

df_workflow: (2491, 4)
   execution_count  companyId        companyName    month
0              258       1794           ARCA ARG  2025-01
1               27       1102            Agritop  2025-01
2               28       1562      

In [9]:
# Pivot billing data to have MARKETING_LITE and UTILITY as separate columns
df_billing_pivot = df_billing.pivot_table(
    index=['companyId', 'month'],
    columns='conversationType',
    values='billable_count',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Flatten column names
df_billing_pivot.columns.name = None

print("df_billing_pivot:", df_billing_pivot.shape)
print(df_billing_pivot.columns.tolist())
df_billing_pivot.head()


df_billing_pivot: (593, 5)
['companyId', 'month', 'MARKETING', 'MARKETING_LITE', 'UTILITY']


,companyId,month,MARKETING,MARKETING_LITE,UTILITY
0,1,2025-06,0,0,4
1,1,2025-07,0,16451,2358
2,1,2025-08,0,20387,3291
3,1,2025-09,0,22725,2993
4,1,2025-10,0,22911,3021


In [10]:
# Merge the three dataframes on companyId and month
df_merged = df_workflow.merge(df_billing_pivot, on=['companyId', 'month'], how='outer')
df_merged = df_merged.merge(df_dau, on=['companyId', 'month'], how='outer')

print("df_merged:", df_merged.shape)
print(f"Unique companies: {df_merged['companyId'].nunique()}")
print(f"Unique months: {df_merged['month'].nunique()}")
print("Columns:", df_merged.columns.tolist())
df_merged.head(10)


df_merged: (2625, 9)
Unique companies: 552
Unique months: 12
Columns: ['execution_count', 'companyId', 'companyName', 'month', 'MARKETING', 'MARKETING_LITE', 'UTILITY', 'dau_count', 'unique_users_count']


,execution_count,companyId,companyName,month,MARKETING,MARKETING_LITE,UTILITY,dau_count,unique_users_count
0,5082.0,1,Shippify,2025-04,NaN,NaN,NaN,NaN,NaN
1,10777.0,1,Shippify,2025-05,NaN,NaN,NaN,14.0,14.0
2,21152.0,1,Shippify,2025-06,0.0,0.0,4.0,NaN,NaN
3,26329.0,1,Shippify,2025-07,0.0,16451.0,2358.0,NaN,NaN
4,28414.0,1,Shippify,2025-08,0.0,20387.0,3291.0,NaN,NaN
5,25384.0,1,Shippify,2025-09,0.0,22725.0,2993.0,NaN,NaN
6,22851.0,1,Shippify,2025-10,0.0,22911.0,3021.0,NaN,NaN
7,22715.0,1,Shippify,2025-11,0.0,77538.0,42.0,NaN,NaN
8,21002.0,1,Shippify,2025-12,0.0,40870.0,1.0,NaN,NaN
9,1008.0,5,DevLab,2025-01,NaN,NaN,NaN,6.0,6.0


In [11]:
# Save the merged dataframe for Streamlit
df_merged.to_csv('data/merged_data.csv', index=False)
print("Saved merged data to data/merged_data.csv")


Saved merged data to data/merged_data.csv
